## Scraping du site Coursera afin de faire des études sur les certifications

In [5]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import pandas as pd
import os
import os
import os
import os

###### bibliothèques utilisées

In [6]:
def scrolling ():
    driver = webdriver.Chrome()
    search_key = input("donner le mot que vous voulez chercher dans Coursera ")
    link = f'https://www.coursera.org/search?query={search_key}'
    driver.get(link)

        # à fixer ces valeurs en prenons en considération la vitesse de chargement des élements de la page

    scroll_pause_time = float(input("donner le scroll pause time (0.5), tout dépendant de votre connexion et PC"))  # Time to wait between scrolls
    scroll_duration = float(input("donner la durée de scroll (20), tout dépendant de votre connexion et PC"))  # Total scroll duration
    scroll_amount = float(input("donner le nombre de pixel à scroller (500), tout dépendant de votre connexion et PC"))  # Pixels to scroll each time


    start_time = time.time()
    while time.time() - start_time < scroll_duration:
        driver.execute_script(f"window.scrollBy(0, {scroll_amount});")
        time.sleep(scroll_pause_time)

        # Get the page source after scrolling
    page_source = driver.page_source
    #time.sleep(10)
    driver.quit()

    return page_source

###### fonction pour faire descendre la page automatiquement et qui retourne en fin le code source HTML de la page

In [7]:
response = scrolling()
soup = BeautifulSoup(response, 'lxml')

all_courses = []
all_names = []
all_levels = []
all_certif_type = []
all_duration = []
all_ratings = []
all_skills = []

In [8]:
outer_div = soup.find_all('ul', class_="cds-9 css-5t8l4v cds-10")
for outer_div_1 in outer_div:
    inner_div = outer_div_1.find_all('li', class_="cds-9 css-0 cds-11 cds-grid-item cds-56 cds-64 cds-76 cds-90")
    for j in inner_div:
            #liste des noms des certifs
        course_name = j.find('h3', class_ ="cds-CommonCard-title css-6ecy9b").text
        all_courses.append(course_name)



            #nom des sociétés
        name = j.find('p', class_="cds-ProductCard-partnerNames css-vac8rf")
        if name:
            all_names.append(name.text)
        else:
            all_names.append("name not found")



            # information sur le niveau, durée,
        levels = j.find('div', class_="cds-CommonCard-metadata")
        level = levels.find('p', class_="css-vac8rf")
        if level:
            # il faut que les informations soient aplaties ou bien dans une liste pour garder une bonne symétrie
            level_in_list = [level.text.split('·')[0]]
            certif_in_list = [level.text.split('·')[1]]
            duration_in_list = [level.text.split('·')[2]]


            all_levels.append(level_in_list)
            all_certif_type.append(certif_in_list)
            all_duration.append(duration_in_list)
        else:
            all_levels.append("no info dispo")



            # Ratings sur la certif
        ratings = j.find('div', class_="cds-CommonCard-ratings")
        if ratings:
            # Locate the nested div with class="cds-RatingStat-meter"
            result = ratings.find('div', class_="cds-RatingStat-meter")
            if result:
                # Extract the aria-valuetext attribute
                aria_value_text = result.get('aria-valuetext')
                if aria_value_text:
                    all_ratings.append(aria_value_text)
                else:
                        print("aria-valuetext attribute not found!")
            else:
                print("Inner div with class 'cds-RatingStat-meter' not found!")
        else:
            print("Outer div with class 'cds-CommonCard-ratings' not found!")


            # Skills que la certif guarantit
        temp_list = []
        result = j.find('div', class_="cds-ProductCard-body")
        if result:
            h = result.find('div', class_="cds-CommonCard-bodyContent")
            skill = h.find('p', class_="css-vac8rf").text.strip()
            temp_list.append(skill)
            all_skills.append(temp_list[0][32::])
        else:
            temp_list.append("No information given")
            all_skills.append(temp_list)


aria-valuetext: 4,6 sur 5 étoiles
aria-valuetext: 4,7 sur 5 étoiles
aria-valuetext: 4,8 sur 5 étoiles
aria-valuetext: 4,6 sur 5 étoiles
aria-valuetext: 4,7 sur 5 étoiles
aria-valuetext: 4,8 sur 5 étoiles
aria-valuetext: 4,6 sur 5 étoiles
aria-valuetext: 4,6 sur 5 étoiles
aria-valuetext: 4,5 sur 5 étoiles
aria-valuetext: 4,7 sur 5 étoiles
aria-valuetext: 4,6 sur 5 étoiles
aria-valuetext: 4,6 sur 5 étoiles
aria-valuetext: 4,8 sur 5 étoiles
aria-valuetext: 4,6 sur 5 étoiles
aria-valuetext: 4,6 sur 5 étoiles
aria-valuetext: 4,7 sur 5 étoiles
aria-valuetext: 4,7 sur 5 étoiles
aria-valuetext: 3,7 sur 5 étoiles
aria-valuetext: 4,7 sur 5 étoiles
aria-valuetext: 4,6 sur 5 étoiles
aria-valuetext: 4,6 sur 5 étoiles
aria-valuetext: 4,8 sur 5 étoiles
aria-valuetext: 4,7 sur 5 étoiles
aria-valuetext: 4,6 sur 5 étoiles
aria-valuetext: 4,5 sur 5 étoiles
aria-valuetext: 3,7 sur 5 étoiles
aria-valuetext: 4,7 sur 5 étoiles
Inner div with class 'cds-RatingStat-meter' not found!
aria-valuetext: 4,6 sur 5 é

###### Scraping et ajout de la data dans des listes à afficher plus tards

In [9]:
min_length = min(len(all_courses),len(all_ratings), len(all_skills), len(all_levels), len(all_certif_type), len(all_duration), len(all_names))
all_courses = all_courses [:min_length]
all_names =  all_names[:min_length]
all_levels = all_levels[:min_length]
all_certif_type = all_certif_type[:min_length]
all_duration = all_duration[:min_length]
all_ratings = all_ratings[:min_length]
all_skills = all_skills[:min_length]

###### Making the arrays the same length to have a beautiful table like you and I :)


In [10]:
file_name = "table.csv"
if os.path.exists(file_name):
    os.remove(file_name)

df = pd.DataFrame({
    "Course Name": all_courses,
    "Certificate Provider": all_names,
    "Certification Level": all_levels,
    "Certification Type": all_certif_type,
    "Duration": all_duration,
    "Certification Rating": all_ratings,
    "Skills": all_skills
})

###### Suppression du fichier s'il existe déjà, afin d'éviter les problèmes de chevauchement.

In [11]:
df.to_csv(file_name, index=False, encoding="utf-8")

print("dataframe saved successfully :) ")

dataframe saved successfully :) 


###### Sauvegarde de la data en un csv file dans le main dossier